# Kn Computation from Measurement-Head CSV

This notebook demonstrates how to:
1. Load a measurement-head geometry CSV file
2. Compute kn (sensitivity coefficients) for a segment
3. Export the kn values to the standard TXT format
4. Visualize the kn magnitude spectrum

## Background

The measurement head CSV contains coil geometry data (radius, position, turns, etc.).
From this geometry, we compute the complex sensitivity coefficients $k_n$ for each harmonic order $n$.

The output format is whitespace-delimited columns:
```
Abs_Re  Abs_Im  Cmp_Re  Cmp_Im  [Ext_Re  Ext_Im]
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from rotating_coil_analyzer.analysis import (
    compute_head_kn_from_csv,
    compute_segment_kn_from_head,
    write_segment_kn_txt,
    load_segment_kn_txt,
    KnBundle,
)

## 1. Load the Measurement-Head CSV

Specify the path to your measurement head CSV file.

In [ ]:
# Path to measurement head CSV
# Adjust this path to point to your MH CSV file
MH_CSV_PATH = Path("../../golden_standards/measurement_heads/CRMMMMH_AF-00000001.csv")

# Verify the file exists
if not MH_CSV_PATH.exists():
    raise FileNotFoundError(f"MH CSV not found: {MH_CSV_PATH}")

print(f"Loading: {MH_CSV_PATH}")

## 2. Compute Head-Level Kn

Parse the CSV and compute per-coil kn values.

In [ ]:
# Computation parameters
N_MULTIPOLES = 15  # Number of harmonic orders to compute
WARM_GEOMETRY = True  # Use warm (room temperature) geometry
USE_DESIGN_RADIUS = True  # Use design radius from CSV

# Compute head-level kn
head_kn = compute_head_kn_from_csv(
    str(MH_CSV_PATH),
    warm_geometry=WARM_GEOMETRY,
    n_multipoles=N_MULTIPOLES,
    use_design_radius=USE_DESIGN_RADIUS,
    strict_header=True,
)

print(f"Source: {head_kn.source_path}")
print(f"Warm geometry: {head_kn.warm_geometry}")
print(f"Use design radius: {head_kn.use_design_radius}")
print(f"Number of multipoles: {head_kn.n_multipoles}")
print(f"Number of coil indices: {len(head_kn.kn_by_index)}")
print(f"\nAvailable coil indices (array.coil): {list(head_kn.kn_by_index.keys())}")

## 3. Compute Segment-Level Kn

Combine coils to form absolute and compensated channels.

### Connection specification format
- Single coil: `"1.3"` means array 1, coil 3
- Difference: `"1.1-1.3"` means (array 1, coil 1) minus (array 1, coil 3)
- Sum: `"1.2+1.4"` means (array 1, coil 2) plus (array 1, coil 4)

### Compensation Scheme (REQUIRED)

**Important:** The compensation scheme label (e.g., "A-C", "ABCD", "none") is NOT inferable from the MH CSV file.
The CSV contains only coil geometry data (radius, angles, turns, etc.), not wiring/connection metadata.

You MUST specify the compensation scheme explicitly below. This label is stored in the KnBundle provenance
and propagated to all downstream exports for traceability.

Common compensation schemes:
- `"none"` or `"single"`: Single coil (no compensation)
- `"A-C"`: Two-coil difference (e.g., coil 1 minus coil 5)
- `"A-B-C-D"` or `"ABCD"`: Four-coil bucking (alternating sum/difference)
- `"custom"`: Non-standard wiring (document in notes)

In [ ]:
# Connection specifications
# These depend on your measurement head wiring
ABS_CONNECTION = "1.3"       # Absolute channel: single center coil
CMP_CONNECTION = "1.1-1.5"   # Compensated channel: A-C scheme (coil 1 minus coil 5)
EXT_CONNECTION = None         # Optional external channel

# REQUIRED: Compensation scheme label for traceability
# This is NOT inferable from the MH CSV - you must specify it explicitly!
COMPENSATION_SCHEME = "A-C"  # <-- Set this to match your wiring (e.g., "A-C", "ABCD", "none")

# Validate that compensation scheme is set
if not COMPENSATION_SCHEME or COMPENSATION_SCHEME == "CHANGE_ME":
    raise ValueError(
        "COMPENSATION_SCHEME must be set explicitly. "
        "The MH CSV does not contain this information."
    )

# Compute segment kn
segment_kn = compute_segment_kn_from_head(
    head_kn,
    abs_connection=ABS_CONNECTION,
    cmp_connection=CMP_CONNECTION,
    ext_connection=EXT_CONNECTION,
    source_label=f"head_csv:{MH_CSV_PATH.name}",
)

print(f"Compensation scheme: {COMPENSATION_SCHEME}")
print(f"Harmonic orders: {segment_kn.orders}")
print(f"kn_abs shape: {segment_kn.kn_abs.shape}")
print(f"kn_cmp shape: {segment_kn.kn_cmp.shape}")
print(f"kn_ext: {'present' if segment_kn.kn_ext is not None else 'None'}")

## 4. Sanity Checks

In [ ]:
# Check for NaN values
nan_abs = np.sum(~np.isfinite(segment_kn.kn_abs))
nan_cmp = np.sum(~np.isfinite(segment_kn.kn_cmp))

print("=== Sanity Checks ===")
print(f"Number of harmonics: {len(segment_kn.orders)}")
print(f"Order range: n = {segment_kn.orders[0]} to {segment_kn.orders[-1]}")
print(f"NaN in kn_abs: {nan_abs}")
print(f"NaN in kn_cmp: {nan_cmp}")

# Check magnitude ranges
mag_abs = np.abs(segment_kn.kn_abs)
mag_cmp = np.abs(segment_kn.kn_cmp)

print(f"\n|kn_abs| range: [{mag_abs.min():.6e}, {mag_abs.max():.6e}]")
print(f"|kn_cmp| range: [{mag_cmp.min():.6e}, {mag_cmp.max():.6e}]")

if nan_abs > 0 or nan_cmp > 0:
    print("\nWARNING: NaN values detected! Check coil connections.")
else:
    print("\nAll checks passed.")

## 5. Visualize Kn Magnitude Spectrum

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

orders = segment_kn.orders

# Linear scale
ax1.plot(orders, np.abs(segment_kn.kn_abs), 'o-', label='|kn_abs|', markersize=6)
ax1.plot(orders, np.abs(segment_kn.kn_cmp), 's-', label='|kn_cmp|', markersize=6)
ax1.set_xlabel('Harmonic order n')
ax1.set_ylabel('|kn|')
ax1.set_title('Kn Magnitude (Linear Scale)')
ax1.legend()
ax1.grid(True)
ax1.set_xticks(orders)

# Log scale
ax2.semilogy(orders, np.abs(segment_kn.kn_abs), 'o-', label='|kn_abs|', markersize=6)
ax2.semilogy(orders, np.abs(segment_kn.kn_cmp), 's-', label='|kn_cmp|', markersize=6)
ax2.set_xlabel('Harmonic order n')
ax2.set_ylabel('|kn| (log scale)')
ax2.set_title('Kn Magnitude (Log Scale)')
ax2.legend()
ax2.grid(True, which='both')
ax2.set_xticks(orders)

plt.tight_layout()
plt.show()

## 6. Export to Standard Kn TXT Format

In [ ]:
# Output path
OUTPUT_PATH = Path("./Kn_values_computed.txt")

# Write the kn file
write_segment_kn_txt(segment_kn, str(OUTPUT_PATH))

print(f"Exported to: {OUTPUT_PATH.absolute()}")

# Show first few lines of the output
print("\nFile contents (first 5 lines):")
with open(OUTPUT_PATH) as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        print(f"  {line.rstrip()}")

## 7. Verify Round-Trip (Optional)

Load the exported file back and verify values match.

In [ ]:
# Reload the exported file
kn_reloaded = load_segment_kn_txt(str(OUTPUT_PATH))

# Compare
abs_diff = np.max(np.abs(kn_reloaded.kn_abs - segment_kn.kn_abs))
cmp_diff = np.max(np.abs(kn_reloaded.kn_cmp - segment_kn.kn_cmp))

print(f"Round-trip verification:")
print(f"  Max |kn_abs| difference: {abs_diff:.2e}")
print(f"  Max |kn_cmp| difference: {cmp_diff:.2e}")

if abs_diff < 1e-12 and cmp_diff < 1e-12:
    print("  PASS: Round-trip successful")
else:
    print("  WARN: Precision loss detected")

## 8. Create KnBundle with Full Provenance (Optional)

For use with the GUI Harmonic Merge tab.

In [ ]:
# Create KnBundle with full provenance metadata
# The compensation_scheme is stored in the 'extra' dict for downstream traceability
bundle = KnBundle(
    kn=segment_kn,
    source_type="head_csv",
    source_path=str(MH_CSV_PATH),
    timestamp=KnBundle.now_iso(),
    segment_id="Main",
    aperture_id=None,
    head_abs_connection=ABS_CONNECTION,
    head_cmp_connection=CMP_CONNECTION,
    head_ext_connection=EXT_CONNECTION,
    head_warm_geometry=WARM_GEOMETRY,
    head_n_multipoles=N_MULTIPOLES,
    extra={"compensation_scheme": COMPENSATION_SCHEME},  # REQUIRED: user-specified
)

print("KnBundle metadata:")
for k, v in bundle.to_metadata_dict().items():
    print(f"  {k}: {v}")